In [19]:
import csv

import catboost
import numpy as np
import numpy.typing as npt

TRAIN_DATA = "../data/train.csv"
VAL_DATA = "../data/val.csv"
SEED = 21

In [16]:
INT_VALUES = ["free sulfur dioxide", "total sulfur dioxide", "quality"]
FLOAT_VALUES = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "density", "pH", "sulphates", "alcohol"]

def read_data(file_name: str) -> tuple[npt.NDArray[npt.NDArray[np.float32]], npt.NDArray[np.int32]]:
    data = []
    labels = []

    with open(TRAIN_DATA, "r", encoding="utf-8") as f:
        csv_reader = csv.reader(f, delimiter=";")
        next(csv_reader)
        for line in csv_reader:
            labels.append(np.int32(line[-1]))
            data.append(np.asarray(line[:-1], dtype=np.float32))
    return np.asarray(data), np.asarray(labels)

(array([[ 7.5  ,  0.29 ,  0.26 , ...,  3.04 ,  0.49 ,  9.2  ],
        [ 5.4  ,  0.205,  0.16 , ...,  3.4  ,  0.38 , 10.8  ],
        [ 7.   ,  0.685,  0.   , ...,  3.34 ,  0.6  ,  9.7  ],
        ...,
        [ 5.7  ,  0.28 ,  0.36 , ...,  3.27 ,  0.98 , 11.9  ],
        [ 6.4  ,  0.15 ,  0.25 , ...,  3.16 ,  0.4  ,  9.9  ],
        [ 7.2  ,  0.25 ,  0.32 , ...,  3.17 ,  0.48 , 11.1  ]],
       dtype=float32),
 array([4, 6, 5, ..., 7, 6, 6], dtype=int32))

In [17]:
train_data, train_labels = read_data(TRAIN_DATA)
val_data, val_labels = read_data(VAL_DATA)

In [20]:
model = catboost.CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    loss_function="Logloss",
    task_type="GPU",
    random_seed=SEED
)
model.fit(train_data, train_labels)

CatBoostError: catboost/libs/train_lib/trainer_env.cpp:9: Environment for task type [GPU] not found